In [6]:
# Import necessary modules and libraries
import os # Interact w/ operating system
from PIL import Image, ImageDraw, ImageFont # Python Imaging Library (image processing)
import numpy as np # Analyze & manipulate arrays
import math # Mathematical constants and functions

In [7]:
# Get all image names from corresponding directory and sort alphabetically
dir = '/content/drive/MyDrive/Eclipse/Images/Cropped'
images = sorted(os.listdir(dir))

# Print all image names
print(images)

['2024-04-08 12:11:31.jpg', '2024-04-08 12:12:13.jpg', '2024-04-08 12:12:30.jpg', '2024-04-08 12:13:53.jpg', '2024-04-08 12:14:45.jpg', '2024-04-08 12:16:29.jpg', '2024-04-08 12:17:37.jpg', '2024-04-08 12:17:51.jpg', '2024-04-08 12:18:34.jpg', '2024-04-08 12:19:20.jpg', '2024-04-08 12:20:22.jpg', '2024-04-08 12:21:31.jpg', '2024-04-08 12:21:35.jpg', '2024-04-08 12:22:24.jpg', '2024-04-08 12:22:46.jpg', '2024-04-08 12:24:46.jpg', '2024-04-08 12:25:44.jpg', '2024-04-08 12:26:02.jpg', '2024-04-08 12:26:54.jpg', '2024-04-08 12:27:11.jpg', '2024-04-08 12:28:59.jpg', '2024-04-08 12:29:48.jpg', '2024-04-08 12:31:06.jpg', '2024-04-08 12:31:22.jpg', '2024-04-08 12:32:59.jpg', '2024-04-08 12:33:04.jpg', '2024-04-08 12:33:10.jpg', '2024-04-08 12:33:31.jpg', '2024-04-08 12:34:32.jpg', '2024-04-08 12:36:23.jpg', '2024-04-08 12:37:51.jpg', '2024-04-08 12:37:52.jpg', '2024-04-08 12:38:18.jpg', '2024-04-08 12:38:20.jpg', '2024-04-08 12:38:21.jpg', '2024-04-08 12:40:46.jpg', '2024-04-08 12:42:08.jpg', 

In [8]:
# Calculate a circle's center using 3 points on its circumference
def center(point1, point2, point3):

  # Get x & y coordinates of each point
  x1, y1 = point1
  x2, y2 = point2
  x3, y3 = point3

  # Get the necessary coefficients to create a circle
  A = x1 * (y2 - y3) - y1 * (x2 - x3) + x2 * y3 - x3 * y2
  B = (x1 ** 2 + y1 ** 2) * (y3 - y2) + (x2 ** 2 + y2 ** 2) * (y1 - y3) + (x3 ** 2 + y3 ** 2) * (y2 - y1)
  C = (x1 ** 2 + y1 ** 2) * (x2 - x3) + (x2 ** 2 + y2 ** 2) * (x3 - x1) + (x3 ** 2 + y3 ** 2) * (x1 - x2)

  # Calculate the center of the circle based on the coefficients
  x = -B / (2 * A)
  y = -C / (2 * A)

  # Return the corresponding coordinates
  return x, y

In [9]:
# Calculate the distance between 2 points
def distance(point1, point2):

  # Separate values
  x1, y1 = point1
  x2, y2 = point2

  # Return the corresponding value using the distance equation
  return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

In [10]:
# Create directory to save images
save_dir = '/content/drive/MyDrive/Eclipse/Images/Modified'
os.mkdir(save_dir)

# Image counter
ic = 0

# For each image
for i in images:

  # Open the image
  image = Image.open(os.path.join(dir, i))

  # Turn the image grayscale
  g_image = image.convert('L')

  # Save placeholders to estimate Sun's position
  left = image.size
  right = (0,0)
  bottom = (0,0)

  # Establish a threshold based on the image's max pixel value
  max = np.max(np.array(g_image)) * 0.81

  # Store the number of pixels with a value greater than the threshold
  p = 0

  # For every pixel in the horizontal direction
  for x in range(g_image.size[0]):

    # For every pixel in the vertical direction
    for y in range(g_image.size[1]):

      # Get the pixel's value
      value = g_image.getpixel((x, y))

      # If the pixel's value is greater or equal to the threshold...
      if value >= max:

        # Update the number of pixels with a value greater than the threshold
        p += 1

        # ...and it's also the leftmost value, save its coordinates
        if x < left[0]:
          left = (x,y)

        # ...and it's also the rightmost value, save its coordinates
        if x > right[0]:
          right = (x,y)

        # ...and it's also the bottommost value, save its coordinates
        if y > bottom[1]:
          bottom = (x,y)

  # Estimate the Sun's center, radius and area
  c = center(left, right, bottom)
  r = distance(c, bottom)
  a = math.pi * r ** 2

  # Get obscuration percentage
  o = int(100-((p/a)*100))

  # Draw the Sun's outline
  draw = ImageDraw.Draw(image)
  draw.ellipse([(c[0]-r, c[1]-r),
                (c[0]+r, c[1]+r)], outline='white')

  # Crop the original image and center it on the Sun
  image = image.crop((int(c[0])-540, int(c[1])-540,
                      int(c[0])+540, int(c[1])+540))

  # Add the image's datetime in the bottom left corner
  font = ImageFont.truetype(r'/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf', 40)
  draw = ImageDraw.Draw(image)
  draw.text((25, 1080-40-25), i[:-4], font=font)

  # Add the obscurity percentage in the bottom right corner
  bounds = draw.textbbox((0,0), f'{o}%', font=font)
  draw.text((1080-bounds[2]-25, 1080-40-25), f'{o}%', font=font)

  # Save the image to the appropriate directory
  image.save(os.path.join(save_dir, i))

  # Print saved image information and upload counter
  print(f'{ic} | {i} | {o}% | {max}')
  ic += 1

0 | 2024-04-08 12:11:31.jpg | 89% | 192.78
1 | 2024-04-08 12:12:13.jpg | 88% | 198.45000000000002
2 | 2024-04-08 12:12:30.jpg | 89% | 196.02
3 | 2024-04-08 12:13:53.jpg | 88% | 196.02
4 | 2024-04-08 12:14:45.jpg | 87% | 196.02
5 | 2024-04-08 12:16:29.jpg | 86% | 196.83
6 | 2024-04-08 12:17:37.jpg | 85% | 198.45000000000002
7 | 2024-04-08 12:17:51.jpg | 85% | 198.45000000000002
8 | 2024-04-08 12:18:34.jpg | 83% | 198.45000000000002
9 | 2024-04-08 12:19:20.jpg | 82% | 199.26000000000002
10 | 2024-04-08 12:20:22.jpg | 82% | 200.07000000000002
11 | 2024-04-08 12:21:31.jpg | 79% | 200.88000000000002
12 | 2024-04-08 12:21:35.jpg | 79% | 198.45000000000002
13 | 2024-04-08 12:22:24.jpg | 78% | 200.07000000000002
14 | 2024-04-08 12:22:46.jpg | 77% | 200.07000000000002
15 | 2024-04-08 12:24:46.jpg | 75% | 200.07000000000002
16 | 2024-04-08 12:25:44.jpg | 74% | 200.07000000000002
17 | 2024-04-08 12:26:02.jpg | 73% | 200.88000000000002
18 | 2024-04-08 12:26:54.jpg | 71% | 200.88000000000002
19 | 2

In [11]:
# Create a list to store all GIF images
gif_images = []

# For each cropped image
for i in images:

  # Open the image
  image = Image.open(os.path.join(save_dir, i))

  # Append the image to the appropriate list
  gif_images.append(image)

# Create a GIF combining all stored images
gif_images[0].save('EclipseTimelapseData.gif', save_all=True, append_images=gif_images[1:], duration=100, loop=0)
gif_images[0].save('/content/drive/MyDrive/Eclipse/EclipseTimelapseData.gif', save_all=True, append_images=gif_images[1:], duration=100, loop=0)